<a href="https://colab.research.google.com/github/VivekGupta773/plagiarism_checker/blob/main/plagiarism_checke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
from collections import Counter

def preprocess_text(text):
    # Remove punctuation and convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    return text

def calculate_similarity(file1, file2):
    # Read the contents of the files
    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        text1 = f1.read()
        text2 = f2.read()

    # Preprocess the texts
    processed_text1 = preprocess_text(text1)
    processed_text2 = preprocess_text(text2)

    # Calculate word frequencies using Counter
    word_freq1 = Counter(processed_text1.split())
    word_freq2 = Counter(processed_text2.split())

    # Calculate cosine similarity
    common_words = set(word_freq1.keys()) & set(word_freq2.keys())
    dot_product = sum(word_freq1[word] * word_freq2[word] for word in common_words)
    magnitude1 = sum(word_freq1[word] ** 2 for word in word_freq1.keys())
    magnitude2 = sum(word_freq2[word] ** 2 for word in word_freq2.keys())
    similarity = dot_product / (magnitude1 ** 0.5 * magnitude2 ** 0.5)

    return similarity

# Get the file paths from the user
file1_path = input("Enter the path to the first file: ")
file2_path = input("Enter the path to the second file: ")

# Calculate similarity
similarity_score = calculate_similarity(file1_path, file2_path)

# Output the similarity score
print(f"The similarity score between the two files is: {similarity_score}")


In [12]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Get a list of all text files in the current directory
student_files = [doc for doc in os.listdir() if doc.endswith('.txt')]

# Read the contents of each student's text file
student_notes = [open(_file, encoding='utf-8').read() for _file in student_files]

# Function to vectorize the text using TF-IDF
def vectorize(Text):
    return TfidfVectorizer().fit_transform(Text).toarray()

# Function to calculate cosine similarity between two documents
def similarity(doc1, doc2):
    return cosine_similarity([doc1, doc2])

# Vectorize the student notes using TF-IDF
vectors = vectorize(student_notes)
s_vectors = list(zip(student_files, vectors))
plagiarism_results = set()

# Function to check plagiarism among the student notes
def check_plagiarism():
    global s_vectors
    for student_a, text_vector_a in s_vectors:
        new_vectors = s_vectors.copy()
        current_index = new_vectors.index((student_a, text_vector_a))
        del new_vectors[current_index]
        for student_b, text_vector_b in new_vectors:
            # Calculate cosine similarity between two text vectors
            sim_score = similarity(text_vector_a, text_vector_b)[0][1]
            # Sort the student file names alphabetically to avoid duplicates
            student_pair = sorted((student_a, student_b))
            # Create a tuple with student file names and similarity score
            score = (student_pair[0], student_pair[1], sim_score)
            # Add the tuple to plagiarism_results set
            plagiarism_results.add(score)
    return plagiarism_results

# Print the plagiarism results
for data in check_plagiarism():
    print("Similarity data:\n", data)


Similarity data:
 ('Arthur.txt', 'Ben.txt', 0.49793145667331107)
